In [1]:
%matplotlib inline

# ALE meta-analysis

In [2]:
import os
import nibabel as nib

import matplotlib.pyplot as plt
from nilearn.plotting import plot_stat_map
from nimare.io import convert_sleuth_to_dataset
from nimare.utils import get_resource_path

from nimare.correct import FWECorrector
from nimare.meta.cbma import ALE

ERROR:duecredit:Failed to import duecredit due to No module named 'duecredit'


In [3]:
_dset_folder = r'../data_db/dataset_ale/'
_nii_folder = r'../data_db/nifti-ale/'

## ALE Working Memory: visual task stimulus and task modality n-task

In [5]:
wm_sel1_file = os.path.join(os.path.abspath(_dset_folder), "select01_wm.txt")
wm_sel1_dset = convert_sleuth_to_dataset(wm_sel1_file)

In [ ]:
wm_sel1_dset.coordinates

In [ ]:
# ale = ALE(null_method="montecarlo",n_iters=100,n_cores=8)
ale = ALE(null_method="approximate")
wm_sel1_results = ale.fit(wm_sel1_dset)
corr = FWECorrector(method="montecarlo", voxel_thresh=0.001, n_iters=100, n_cores=8)
wm_sel1_corrected_results = corr.transform(wm_sel1_results)

In [ ]:
map_name = "z_desc-size_level-cluster_corr-FWE_method-montecarlo"
fig, axes = plt.subplots(figsize=(12, 10), nrows=1)
wm_sel1_img = wm_sel1_corrected_results.get_map(
    map_name
)

plot_stat_map(
    wm_sel1_img,
    cut_coords=6,
    display_mode="z",
    title="Semantic select1 - wm",
    threshold=2.326,  #'auto' # cluster-level p < .01, one-tailed
    cmap="RdBu_r",
    vmax=4,
    axes=axes,
    figure=fig,
)


## Save img to nifti

In [ ]:
img_name = 'img_wm_sel1'
nib.save(wm_sel1_img, os.path.join(os.path.abspath(_nii_folder), img_name + '.nii'))
nib.save(wm_sel1_corrected_results.get_map('z'), os.path.join(os.path.abspath(_nii_folder), img_name + '-z.nii'))

## Statistics

In [ ]:
from nimare.diagnostics import FocusCounter

counter = FocusCounter(
    target_image="z_desc-size_level-cluster_corr-FWE_method-montecarlo",
    voxel_thresh=None,
)
test1_count_table, _ = counter.transform(wm_sel1_corrected_results)
test1_count_table.shape

In [ ]:
test1_count_table

In [ ]:
from nimare.diagnostics import Jackknife

jackknife = Jackknife(
    target_image="z_desc-size_level-cluster_corr-FWE_method-montecarlo",
    voxel_thresh=None,
)
test2_jackknife_table, _ = jackknife.transform(wm_sel1_corrected_results)
test2_jackknife_table.head(10)